In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from matplotlib.pyplot import figure
import matplotlib.dates as mdates
import datetime
import seaborn as sns

In [29]:
df_validation = pd.read_csv(r'../../data/tidy/energy_validation_april2022.csv', delimiter=',', skiprows=0, low_memory=False)

In [30]:
df_model = pd.read_csv(r'../../results/computed-fuel-rates.csv', delimiter=',', skiprows=0, low_memory=False)

In [31]:
df_validation

,Unnamed: 0,timestamp,equipment_id,gallons,date
0,58932,2022-04-01 00:04:00,3213,46.9,2022-04-01
1,58933,2022-04-01 00:11:00,3303,16.0,2022-04-01
2,58934,2022-04-01 00:19:00,3322,31.7,2022-04-01
3,58935,2022-04-01 00:28:00,3312,35.2,2022-04-01
4,58936,2022-04-01 00:33:00,3307,33.4,2022-04-01
...,...,...,...,...,...
2997,4682,4/26/2022 19:12,7816,51.6,2022-04-26
2998,4683,4/27/2022 20:39,7816,19.3,2022-04-27
2999,4684,4/28/2022 14:02,7816,14.0,2022-04-28
3000,4685,4/29/2022 19:10,7816,18.4,2022-04-29


In [32]:
df_model

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0,ServiceDateTime,Unnamed: 0.1,Unnamed: 0.1.1,DateKey,CalendarDate,Year,...,geometry,dist,Speed,Acceleration,time_delta,time_delta_in_seconds,Powertrain,VehicleModel,Vehicle_mass,FuelRate/Energy
0,0,1297419,1297419,2391,2022-04-29 14:01:44,2391,1273034,8788,2022-04-29 00:00:00.000,2022,...,POINT (-72.52602 42.14098),0.000000,0.000000,0.000000,0,0.0,electric,New Flyer 35 electric,26900.0,0
1,1,1299038,1299038,2392,2022-04-29 14:28:20,2392,1273037,8788,2022-04-29 00:00:00.000,2022,...,POINT (-72.65278 42.19514),8.829565,32.052117,0.005579,0 days 00:26:36,1596.0,electric,New Flyer 35 electric,26900.0,10.919433141596903
2,2,1299039,1299039,2393,2022-04-29 14:28:20,2393,1273500,8788,2022-04-29 00:00:00.000,2022,...,POINT (-72.65278 42.19514),0.000000,0.000000,0.000000,0 days 00:00:00,0.0,electric,New Flyer 35 electric,26900.0,0.0
3,3,1299250,1299250,2394,2022-04-29 14:30:55,2394,1273501,8788,2022-04-29 00:00:00.000,2022,...,POINT (-72.64553 42.20302),0.526585,19.682810,0.035274,0 days 00:02:35,155.0,electric,New Flyer 35 electric,26900.0,0.8072716996929717
4,4,1299272,1299272,2395,2022-04-29 14:31:09,2395,1273502,8788,2022-04-29 00:00:00.000,2022,...,POINT (-72.64369 42.20166),0.130185,53.874613,0.678410,0 days 00:00:14,14.0,electric,New Flyer 35 electric,26900.0,1.329511316146411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127243,1127243,1361526,1361526,1361544,2022-04-30 23:54:20,1361544,1325566,8789,2022-04-30 00:00:00.000,2022,...,POINT (-72.53185 42.38201),0.254140,10.517075,0.003673,0 days 00:02:20,140.0,conventional,0,0.0,0
1127244,1127244,1361530,1361530,1361545,2022-04-30 23:55:24,1361545,1325567,8789,2022-04-30 00:00:00.000,2022,...,POINT (-72.5327 42.375),0.156799,14.194300,0.015960,0 days 00:01:04,64.0,conventional,0,0.0,0
1127245,1127245,1361539,1361539,1361546,2022-04-30 23:56:47,1361546,1325568,8789,2022-04-30 00:00:00.000,2022,...,POINT (-72.53357 42.36864),0.144950,10.117888,-0.013643,0 days 00:01:23,83.0,conventional,0,0.0,0
1127246,1127246,1361541,1361541,1361547,2022-04-30 23:57:32,1361547,1325569,8789,2022-04-30 00:00:00.000,2022,...,POINT (-72.53556 42.3656),0.151259,19.474153,0.057755,0 days 00:00:45,45.0,conventional,0,0.0,0


In [33]:
df_model = df_model.fillna(0)
df_model['FuelRate/Energy'] = pd.to_numeric(df_model['FuelRate/Energy'])
df_model_diesel = df_model[df_model.Powertrain != 'electric']
df_daily_fuel = df_model.groupby('Date').agg({'FuelRate/Energy':'sum'})[['FuelRate/Energy']].reset_index()

ValueError: Unable to parse string "1514      -0.000000
2340      -0.000000
2341      -0.000003
2401      -0.078061
2479      -0.028958
             ...   
1319911   -0.261029
1319922   -0.286338
1319927   -0.262999
1320030   -0.000000
1320031   -0.137818
Name: Acceleration, Length: 27848, dtype: float64" at position 5

In [36]:
# pd.to_numeric(df_model['FuelRate/Energy'], errors='coerce')
df_model[pd.to_numeric(df_model['FuelRate/Energy'], errors='coerce').isnull()]

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0,ServiceDateTime,Unnamed: 0.1,Unnamed: 0.1.1,DateKey,CalendarDate,Year,...,geometry,dist,Speed,Acceleration,time_delta,time_delta_in_seconds,Powertrain,VehicleModel,Vehicle_mass,FuelRate/Energy
5,5,1299287,1299287,2396,2022-04-29 14:31:19,2396,1273503,8788,2022-04-29 00:00:00.000,2022,...,POINT (-72.64275 42.20125),0.065495,37.945525,-0.442476,0 days 00:00:10,10.0,electric,New Flyer 35 electric,26900.0,1514 -0.000000\n2340 -0.000000\n2341...
6,6,1299367,1299367,2397,2022-04-29 14:32:20,2397,1273504,8788,2022-04-29 00:00:00.000,2022,...,POINT (-72.64718 42.19427),0.338205,32.121814,-0.026520,0 days 00:01:01,61.0,electric,New Flyer 35 electric,26900.0,1514 0.000000\n2340 0.000000\n2341...
8,8,1299408,1299408,2399,2022-04-29 14:32:53,2399,1273506,8788,2022-04-29 00:00:00.000,2022,...,POINT (-72.64974 42.18943),0.111486,33.995173,-0.057060,0 days 00:00:19,19.0,electric,New Flyer 35 electric,26900.0,1514 0.000000e+00\n2340 0.000000e+...
9,9,1299531,1299531,2400,2022-04-29 14:34:23,2400,1273507,8788,2022-04-29 00:00:00.000,2022,...,POINT (-72.65006 42.18523),0.089318,5.749719,-0.087178,0 days 00:01:30,90.0,electric,New Flyer 35 electric,26900.0,1514 -0.000000e+00\n2340 -0.000000e+...
11,11,1299786,1299786,2402,2022-04-29 14:37:27,2402,1273509,8788,2022-04-29 00:00:00.000,2022,...,POINT (-72.64783 42.17399),0.150683,5.705886,-0.034161,0 days 00:02:33,153.0,electric,New Flyer 35 electric,26900.0,1514 0.000000e+00\n2340 0.000000e+...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27837,27837,1319742,1319742,33206,2022-04-29 20:20:23,33206,1301435,8788,2022-04-29 00:00:00.000,2022,...,POINT (-72.62294 42.1068),0.061199,6.009550,-0.133449,0 days 00:00:59,59.0,electric,New Flyer 40 electric,32770.0,1514 -0.000000e+00\n2340 -0.000000e+...
27840,27840,1319842,1319842,33209,2022-04-29 20:22:47,33209,1301438,8788,2022-04-29 00:00:00.000,2022,...,POINT (-72.61395 42.1088),0.275762,34.731764,-0.086748,0 days 00:00:46,46.0,electric,New Flyer 40 electric,32770.0,1514 0.000000e+00\n2340 0.000000e+...
27843,27843,1319911,1319911,33212,2022-04-29 20:24:34,33212,1301441,8788,2022-04-29 00:00:00.000,2022,...,POINT (-72.60081 42.10913),0.188877,21.456513,-0.295809,0 days 00:00:51,51.0,electric,New Flyer 40 electric,32770.0,1514 -0.000000\n2340 -0.000000\n2341...
27845,27845,1319927,1319927,33214,2022-04-29 20:24:53,33214,1301443,8788,2022-04-29 00:00:00.000,2022,...,POINT (-72.5986 42.10695),0.067128,43.212870,-0.312729,0 days 00:00:09,9.0,electric,New Flyer 40 electric,32770.0,1514 -0.000000\n2340 -0.000000\n2341...


In [ ]:
DataFrame. astype()

In [11]:
df_daily_fuel.plot(x="Date", y="FuelRate/Energy", color='teal')
plt.ylabel("Fuel consumption (gallons)", fontsize=16)
plt.xlabel("Day", fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
#plt.savefig(r'../../figures/fuel-consumption-model.png', dpi=400, bbox_inches='tight')
plt.show()

TypeError: no numeric data to plot